In [ ]:
import numpy as np
#import lightgbm as lgb
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")

In [ ]:
y = train.Pawpularity.copy()
train.drop(["Id", "Pawpularity"], axis=1, inplace = True)

In [ ]:
bins = pd.cut(y, bins = 5)

In [ ]:
kfold = KFold(n_splits = 5)

In [ ]:
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
preds = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
preds['Pawpularity'] = 0

In [ ]:
for fold, (train_idx, val_idx) in enumerate(kfold.split(train)):
    reg = CatBoostRegressor(cat_features = train.columns.tolist(), random_state = 42, verbose = 0)
    reg.fit(train.loc[train_idx], y.loc[train_idx])
    val_p = reg.predict(train.loc[val_idx])
    print(f"fold {fold} validation error: {np.sqrt(mean_squared_error(y.loc[val_idx], val_p))}")    
    preds["Pawpularity"] += reg.predict(test[train.columns]) / 5

In [ ]:
preds.to_csv("submission.csv", index = False)